#### 1. Import pandas library

In [1]:
import pandas as pd

#### 2. Import pymysql and sqlalchemy as you have learnt in the lesson of importing/exporting data 


In [2]:
import pymysql
import sqlalchemy as db
import getpass

#### 3. Create a mysql engine to set the connection to the server. Check the connection details in [this link](https://relational.fit.cvut.cz/search?tableCount%5B%5D=0-10&tableCount%5B%5D=10-30&dataType%5B%5D=Numeric&databaseSize%5B%5D=KB&databaseSize%5B%5D=MB)

In [3]:
#hostname: relational.fit.cvut.cz
#port: 3306
#username: guest
password="relational"


engine = db.create_engine('mysql+pymysql://guest:{}@relational.fit.cvut.cz/stats'.format(password))

#### 4. Import the users table 

In [4]:
query="Select * from users"

df_users=pd.read_sql_query(query, engine)

#### 5. Rename Id column to userId

In [5]:
print(df_users.columns)
df_users=df_users.rename(columns={'Id':'userID'})
print(df_users.columns)

Index(['Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate',
       'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes',
       'AccountId', 'Age', 'ProfileImageUrl'],
      dtype='object')
Index(['userID', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate',
       'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes',
       'AccountId', 'Age', 'ProfileImageUrl'],
      dtype='object')


#### 6. Import the posts table. 

In [6]:
query="Select * from posts"

df_post=pd.read_sql_query(query, engine)

#### 7. Rename Id column to postId and OwnerUserId to userId

In [7]:
print(df_post.columns)
df_post=df_post.rename(columns={'Id':'postId','OwnerUserId':'userID'})
print(df_post.columns)

Index(['Id', 'PostTypeId', 'AcceptedAnswerId', 'CreaionDate', 'Score',
       'ViewCount', 'Body', 'OwnerUserId', 'LasActivityDate', 'Title', 'Tags',
       'AnswerCount', 'CommentCount', 'FavoriteCount', 'LastEditorUserId',
       'LastEditDate', 'CommunityOwnedDate', 'ParentId', 'ClosedDate',
       'OwnerDisplayName', 'LastEditorDisplayName'],
      dtype='object')
Index(['postId', 'PostTypeId', 'AcceptedAnswerId', 'CreaionDate', 'Score',
       'ViewCount', 'Body', 'userID', 'LasActivityDate', 'Title', 'Tags',
       'AnswerCount', 'CommentCount', 'FavoriteCount', 'LastEditorUserId',
       'LastEditDate', 'CommunityOwnedDate', 'ParentId', 'ClosedDate',
       'OwnerDisplayName', 'LastEditorDisplayName'],
      dtype='object')


#### 8. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userID,ViewCount,CommentCount

In [8]:
df_users_cut=df_users[["userID", "Reputation","Views","UpVotes","DownVotes"]]
df_post_cut=df_post[["postId", "Score","userID","ViewCount","CommentCount"]]
df_users_cut.head()


,userID,Reputation,Views,UpVotes,DownVotes
0,-1,1,0,5007,1920
1,2,101,25,3,0
2,3,101,22,19,0
3,4,101,11,0,0
4,5,6792,1145,662,5


In [9]:
df_post_cut.head()

,postId,Score,userID,ViewCount,CommentCount
0,1,23,8.0,1278.0,1
1,2,22,24.0,8198.0,1
2,3,54,18.0,3613.0,4
3,4,13,23.0,5224.0,2
4,5,81,23.0,NaN,3


#### 8. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [10]:
df_merge=df_post_cut.merge(df_users_cut,how="inner",on='userID',suffixes=('_post', '_user'))
df_merge.head()

,postId,Score,userID,ViewCount,CommentCount,Reputation,Views,UpVotes,DownVotes
0,1,23,8.0,1278.0,1,6764,1089,604,25
1,16,16,8.0,NaN,3,6764,1089,604,25
2,36,41,8.0,67396.0,7,6764,1089,604,25
3,65,14,8.0,NaN,3,6764,1089,604,25
4,78,33,8.0,NaN,4,6764,1089,604,25


#### 9. How many missing values do you have in your merged dataframe? On which columns?

In [11]:
print("El tamaño de los datos es",df_merge.shape)
df_merge.isnull().sum()

El tamaño de los datos es (90584, 9)


postId              0
Score               0
userID              0
ViewCount       48396
CommentCount        0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
dtype: int64

#### 10. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [12]:
#There are 48396 values in column "ViewCount" that are NaN. 
#This column could be interpretated as the number of times a pst has been seen.
# The NaN values can be changed by 0.

df_merge["ViewCount"]=df_merge["ViewCount"].fillna(0)
df_merge.isnull().sum()

postId          0
Score           0
userID          0
ViewCount       0
CommentCount    0
Reputation      0
Views           0
UpVotes         0
DownVotes       0
dtype: int64

#### 11. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [13]:
df_merge.head()

,postId,Score,userID,ViewCount,CommentCount,Reputation,Views,UpVotes,DownVotes
0,1,23,8.0,1278.0,1,6764,1089,604,25
1,16,16,8.0,0.0,3,6764,1089,604,25
2,36,41,8.0,67396.0,7,6764,1089,604,25
3,65,14,8.0,0.0,3,6764,1089,604,25
4,78,33,8.0,0.0,4,6764,1089,604,25


In [14]:
df_merge=df_merge.astype(dtype='int64')

df_merge.dtypes


postId          int64
Score           int64
userID          int64
ViewCount       int64
CommentCount    int64
Reputation      int64
Views           int64
UpVotes         int64
DownVotes       int64
dtype: object

In [15]:
df_merge.head()

,postId,Score,userID,ViewCount,CommentCount,Reputation,Views,UpVotes,DownVotes
0,1,23,8,1278,1,6764,1089,604,25
1,16,16,8,0,3,6764,1089,604,25
2,36,41,8,67396,7,6764,1089,604,25
3,65,14,8,0,3,6764,1089,604,25
4,78,33,8,0,4,6764,1089,604,25


#### Bonus: Identify extreme values in your merged dataframe as you have learned in class, create a dataframe called outliers with the same columns as our data set and calculate the bounds. The values of the outliers dataframe will be the values of the merged_df that fall outside that bounds. You will need to save your outliers dataframe to a csv file on your-code folder.

In [16]:
df_clean=df_merge
stats = df_clean.describe().transpose()
stats['IQR'] = stats['75%'] - stats['25%']
stats

,count,mean,std,min,25%,50%,75%,max,IQR
postId,90584.0,56539.080522,33840.307529,1.0,26051.75,57225.5,86145.25,115378.0,60093.50
Score,90584.0,2.780767,4.948922,-19.0,1.00,2.0,3.00,192.0,2.00
userID,90584.0,16546.764727,15273.367108,-1.0,3437.00,11032.0,27700.00,55746.0,24263.00
ViewCount,90584.0,259.253400,1632.261405,0.0,0.00,0.0,111.00,175495.0,111.00
CommentCount,90584.0,1.894650,2.638704,0.0,0.00,1.0,3.00,45.0,3.00
Reputation,90584.0,6282.395412,15102.268670,1.0,60.00,396.0,4460.00,87393.0,4400.00
Views,90584.0,1034.245176,2880.074012,0.0,5.00,45.0,514.25,20932.0,509.25
UpVotes,90584.0,734.315718,2050.869327,0.0,1.00,22.0,283.00,11442.0,282.00
DownVotes,90584.0,33.273249,134.936435,0.0,0.00,0.0,8.00,1920.0,8.00


In [17]:
df_clean.columns.drop(['postId', 'userID'])

Index(['Score', 'ViewCount', 'CommentCount', 'Reputation', 'Views', 'UpVotes',
       'DownVotes'],
      dtype='object')

In [22]:
outliers = pd.DataFrame(columns=df_clean.columns.drop(['postId', 'userID']))

for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = df_clean[(df_clean[col] < lower) | 
                   (df_clean[col] > upper)].copy()
    results['Outlier'] = col
    outliers = outliers.append(results,sort=False)

In [19]:
outliers.to_csv("outliers.csv")

In [23]:
outliers

,Score,ViewCount,CommentCount,Reputation,Views,UpVotes,DownVotes,postId,userID,Outlier
0,23,1278,1,6764,1089,604,25,1.0,8.0,Score
1,16,0,3,6764,1089,604,25,16.0,8.0,Score
2,41,67396,7,6764,1089,604,25,36.0,8.0,Score
3,14,0,3,6764,1089,604,25,65.0,8.0,Score
4,33,0,4,6764,1089,604,25,78.0,8.0,Score
5,8,0,3,6764,1089,604,25,111.0,8.0,Score
6,30,1220,1,6764,1089,604,25,114.0,8.0,Score
9,17,1022,1,6764,1089,604,25,168.0,8.0,Score
10,56,7733,1,6764,1089,604,25,170.0,8.0,Score
12,13,0,7,6764,1089,604,25,262.0,8.0,Score
